In [2]:
import re
import warnings
import pandas as pd
import plotly.graph_objects as go
import plotly
import math

from plotly.subplots import make_subplots

warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.display.max_columns = 300

## Среднемесячная номинальная начисленная заработная плата

В ноутбуке данные загружаются из оригинального файла с сайта Росстата. В приложении streamlit используется БД.

Загружаем данные с двух листов из файла, исправялем названия колонок, убираем строки без данных:

In [3]:
new_data = pd.read_excel('tab3-zpl_2023.xlsx', sheet_name='с 2017 г.', header=4)
old_data = pd.read_excel('tab3-zpl_2023.xlsx', sheet_name='2000-2016 гг.', header=2)

new_data = new_data.set_axis(['Отрасль'] + [str(x) for x in range(2017, 2024)], axis='columns')
old_data = old_data.set_axis(['Отрасль'] + [str(x) for x in range(2000, 2017)], axis='columns')

new_data = new_data.dropna(how='any')
old_data = old_data.dropna(how='any')

Далее нужно совместить виды деятельности с листов 2000-2016 и 2017-2023, чтобы получить полную картину. На листе 2017-2023 более мелкое дробление видов, поэтому совместить их так просто не получится. Но у некоторых видов просто изменилось название, будем работать с ними:

In [4]:
col_name_mapping = {
    'Рыболовство, рыбоводство': 'Рыболовство и рыбоводство',
    'Производство кожи, изделий из кожи и производство обуви': 'Производство кожи и изделий из кожи',
    'Производство резиновых и пластмассовых изделий': 'Производство резиновых и пластмассовых изделий',
    'Торговля оптовая и розничная; ремонт автотранспортных средств и мотоциклов': 'Оптовая и розничная торговля; ремонт автотранспортных средств, мотоциклов, бытовых изделий и предметов личного пользования',
    'Деятельность финансовая и страховая': 'Финансовая деятельность',
    'Государственное управление и обеспечение военной безопасности; социальное обеспечение': 'Государственное управление и обеспечение военной безопасности; социальное страхование',
    'Деятельность в области здравоохранения и социальных услуг': 'Здравоохранение и предоставление социальных услуг',
    'Всего': 'Средняя',
    'Всего по экономике': 'Средняя',
}

def sanitaze_str(str):
    str = str.strip().capitalize()
    str = re.sub(r'\s{2,}', ' ', str)
    if str in col_name_mapping:
        str = col_name_mapping[str]
    return str

new_data['Отрасль'] = new_data['Отрасль'].apply(sanitaze_str)
old_data['Отрасль'] = old_data['Отрасль'].apply(sanitaze_str)
data = old_data.merge(new_data, on='Отрасль')

data

,Отрасль,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Средняя,2223.4,3240.4,4360.3,5498.5,6739.5,8554.9,10633.9,13593.4,17290.1,18637.5,20952.2,23369.2,26628.9,29792.0,32495.0,34030.0,36709.0,39167.0,43724.0,47866.8,51344.0,57243.7,65338.3,73709.3
1,Рыболовство и рыбоводство,2845.6,3839.3,5031.3,5444.5,7084.9,10233.5,12310.8,14797.0,19498.9,22913.5,23781.9,25939.9,29201.4,32437.3,37062.0,46676.0,54927.0,68032.0,75766.0,88603.8,94983.0,101547.4,121726.2,139520.0
2,Добыча полезных ископаемых,5940.2,9099.2,11080.9,13912.4,16841.7,19726.9,23145.2,28107.5,33206.1,35363.4,39895.0,45132.0,50400.6,54161.2,58959.0,63695.0,69936.0,74474.1,83178.0,89343.7,95359.0,103473.5,118375.7,130825.6
3,Обрабатывающие производства,2365.2,3446.6,4439.1,5603.4,6848.9,8420.9,10198.5,12878.7,16049.9,16583.1,19078.0,21780.8,24511.7,27044.5,29511.0,31910.0,34592.0,38501.5,40722.0,43855.0,46510.0,52409.6,60438.7,71273.3
4,Производство кожи и изделий из кожи,1347.8,1986.1,2621.1,3230.0,3774.7,4695.3,5649.1,7537.0,9522.3,10073.2,11345.8,12350.9,13135.7,14725.1,16119.0,17796.0,19847.0,20193.0,23576.0,25680.3,26913.0,31771.2,36445.0,44039.1
5,Производство кокса и нефтепродуктов,4916.3,7012.4,9625.3,11879.0,13729.3,19397.1,22319.6,28565.0,34912.5,37963.7,41563.4,48462.6,59195.0,64760.1,75517.0,81605.0,88194.0,95956.7,87591.0,81685.2,82106.0,87214.4,97720.3,109541.0
6,Производство резиновых и пластмассовых изделий,2140.4,3032.2,3957.1,4950.7,5956.8,6879.2,8767.7,11082.6,13464.0,13850.6,15766.4,17713.3,19758.3,21599.3,23270.0,25006.0,27565.0,30621.8,33581.0,35623.4,39662.0,44817.4,51471.0,61423.1
7,Строительство,2639.8,3859.3,4806.9,6176.7,7304.7,9042.8,10869.2,14333.4,18574.0,18122.2,21171.7,23682.0,25950.6,27701.4,29354.0,29960.0,32332.0,33677.9,38518.0,42629.9,44738.0,51944.2,60849.5,68944.4
8,Оптовая и розничная торговля; ремонт автотранс...,1584.5,2294.9,3068.9,3974.2,4906.2,6552.1,8234.9,11476.3,14927.4,15958.6,18405.9,19613.2,21633.8,23167.8,25601.0,26947.0,30030.0,32092.5,35444.0,40137.0,41867.0,50388.8,57806.1,63572.2
9,Финансовая деятельность,5232.2,8885.2,13245.9,15561.2,17383.8,22463.5,27885.5,34879.8,41871.8,42372.9,50120.0,55788.9,58999.2,63333.0,68565.0,70088.0,80289.0,84904.0,91070.0,103667.8,112680.0,130223.3,146637.1,169105.7


Построим графики среднемесячной з/п для видов деятельности 'Образование', 'Финансовая деятельность', 'Добыча полезных ископаемых':

In [5]:
def get_line(data, name):
    line = data[data['Отрасль'] == name].drop(['Отрасль'], axis=1)
    line = line.squeeze()
    line.index = line.index.map(int)
    return line

# выбранные отрасли + среднее по всем отраслям
lines = ['Образование', 'Финансовая деятельность', 'Строительство', 'Средняя']

fig = go.Figure()
for name in lines:
    dt = get_line(data, name)
    fig.add_trace(go.Scatter(x=dt.index, y=dt.array, name=name))
fig.update_layout(title='Среднемесячная номинальная начисленная заработная плата',
                  xaxis_title='год', yaxis_title='з/п, руб.',
                  margin=dict(l=20, r=10, t=40, b=10))

Во сколько раз выросла з/п за эти годы:

In [6]:
grow_nom = data[data['Отрасль'] == 'Средняя']
grow_nom = (grow_nom['2023'] / grow_nom['2000']).iloc[0]
grow_nom

33.151614644238556

Наименьшие и наибольшие з/п:

In [7]:
new_data.loc[new_data['2023'].idxmin()].iloc[0], new_data.loc[new_data['2023'].idxmax()].iloc[0]

('Производство одежды', 'Добыча нефти и природного газа')

In [8]:
minmax = new_data[['Отрасль', '2023']]
minmax = minmax.set_index('Отрасль')
minmax = minmax['2023'].sort_values()

fig = go.Figure()
fig.add_trace(go.Bar(x=minmax.index, y=minmax.array))
fig.update_layout(title='З/п на 2023 г. по отраслям', xaxis_visible=False,
                  margin=dict(l=20,r=20,b=10,t=40))

### Выводы
1. З/п по всем отраслям растут. Без учета инфляции, рост номинальной средней з/п более чем в 33 раза за 2000-2023 гг.
2. Наименьшие з/п в легкой промышленности (производство одежды), наибольшие - в добыче нефти и газа.

## Инфляция и реальная з/п
Загружаем данные по инфляции, оставляем только колонку с годовой инфляцией:

In [9]:
infl = pd.read_excel('inflation.xlsx', index_col='Год')
infl = infl['Всего']
infl.name = 'Инфляция'
infl = infl.iloc[1:]
infl = infl.sort_index()

Учитываем инфляцию:

In [10]:
data_real = data.copy()
for year in range(2000, 2023+1):
    data_real[str(year)] /= (1 + infl[year] / 100.0)

color, colors = 0, plotly.colors.qualitative.Plotly
fig = go.Figure()
for name in lines:
    dt = get_line(data, name)
    dt_real = get_line(data_real, name)
    fig.add_trace(go.Scatter(x=dt_real.index, y=dt_real.array, name=name + ' с учетом инфл.', line=dict(color=colors[color])))
    fig.add_trace(go.Scatter(x=dt.index, y=dt.array, name=name, line=dict(dash='dot', color=colors[color])))
    color += 1
fig.update_layout(xaxis_title='год', yaxis_title='з/п, руб.',
                  margin=dict(l=20,r=20,b=10,t=40))

Более наглядно изменение з/п показывает ее дисконтирование к ценам 2000-го (или 2023-го) годов:

In [11]:
# старые цены в новые
def compound(year_from, year_to, sum):
    result = sum
    for x in range(year_from, year_to):
        result *= 1.0 + infl.loc[x] / 100.0
    return result

# новые цены в старые
def discount(year_from, year_to, sum):
    result = sum
    for x in range(year_from, year_to, -1):
        result /= 1.0 + infl.loc[x] / 100.0
    return result


In [12]:
data_2023 = data.copy()
data_2000 = data.copy()

for year in range(2000, 2023+1):
    data_2023[str(year)] = compound(year, 2023, data_2023[str(year)])
    data_2000[str(year)] = discount(year, 2000, data_2000[str(year)])

def plot(data, fig, year, col):
    for name in lines:
        dt = get_line(data, name)
        fig.add_trace(go.Scatter(x=dt.index, y=dt.array, name=name, legendgroup = str(col)), col=col, row=1)

titles = [
    'Среднемесячная реальная начисленная<br>заработная плата в ценах 2000 г.',
    'Среднемесячная реальная начисленная<br>заработная плата в ценах 2023 г.',
]

fig = make_subplots(rows=1, cols=2, subplot_titles=titles)
plot(data_2000, fig, 2000, 1)
plot(data_2023, fig, 2023, 2)
fig.update_layout(legend_tracegroupgap = 50, margin=dict(l=20,r=20,b=40,t=50))

Во сколько раз выросла реальная з/п за эти годы:

In [13]:
grow_real = data_2000[data_2000['Отрасль'] == 'Средняя']
grow_real = (grow_real['2023'] / grow_real['2000']).iloc[0]
grow_real

4.458280785233497

### Влияние инфляции на изменение з/п
Сравним изменение з/п год к году с уровнем инфляции:

In [14]:
dt = data_real.transpose()
dt = dt.set_axis(dt.iloc[0], axis='columns')
dt = dt.iloc[1:]
dt.index.name = 'Год'
dt.index = dt.index.map(int)
dt = dt.pct_change() * 100.0
dt = dt.iloc[1:]
dt.join(infl)[lines + ['Инфляция']].transpose()


Год,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
Образование,49.817919,64.585603,18.861661,24.396280,30.140681,30.866676,22.479267,27.313254,22.304746,5.898988,15.155913,19.613081,23.643994,5.387577,2.692519,11.761168,10.729479,11.655608,9.166762,4.827561,6.153205,8.027423,16.809132
Финансовая деятельность,72.137659,53.638954,20.699852,11.962403,30.187914,26.312177,21.873255,18.551770,5.363673,18.304895,14.122271,5.278089,7.476617,3.487731,0.817982,22.740134,8.698036,5.462099,15.180756,6.766269,11.858639,9.052765,20.153641
Строительство,48.193971,28.364113,32.019033,18.526771,24.720689,22.303493,28.488571,27.972489,1.585056,16.848907,14.682279,9.085919,6.877027,1.293602,0.663340,15.628522,7.068577,12.452198,11.985544,3.084498,12.379777,13.449208,18.049604
Средняя,47.731819,38.677123,29.560610,22.844022,27.886653,26.479996,24.551334,25.611617,12.231430,12.440253,14.353085,13.435516,12.015077,4.263746,3.286182,15.580566,9.672401,9.761189,10.770952,5.362584,7.910988,10.540557,17.537652
Инфляция,18.580000,15.060000,11.990000,11.740000,10.910000,9.000000,11.870000,13.280000,8.800000,8.780000,6.100000,6.580000,6.450000,11.360000,12.910000,5.380000,2.520000,4.270000,3.050000,4.910000,8.390000,11.920000,7.420000


Отобразим года, в которые з/п росла быстрее или медленнее инфляции.

In [15]:
n = 0
fig = make_subplots(rows=math.ceil(len(lines) / 2.0), cols=2, subplot_titles=lines)
for line in lines:
    bar = dt[line] - infl[infl.index >= 2001]
    row = n // 2 + 1
    col = n % 2 + 1
    fig.add_trace(go.Bar(x=bar.index, y=bar.array, name=line, marker_color='blue'), col=col, row=row)
    n += 1
fig.update_layout(showlegend=False, margin=dict(l=20, r=20, t=30, b=30))

Примерно до кризиса 2008 г. наблюдался рост реальной з/п даже в условиях высокой (двузначной) инфляции. После этого наблюдается замедление роста или даже снижение реальной з/п вслед за высокой инфляцией.

In [16]:
# корреляция изменения з/п с инфляцией предыдущего года
dt[dt.index < 2007]['Средняя'].corr(infl.shift(1)), dt[dt.index > 2008]['Средняя'].corr(infl.shift(1))

(0.9388658413870828, 0.41659634773393406)

### Выводы
1. Средняя реальная з/п по всем отраслям выросла в ~4.5 раза за 2000-2023.
2. Рост номинальной средней (без учета инфляции) з/п отличается от скорректированного показателя более чем в 7 раз. Накопленная инфляция оказывает значительный эффект на итоговые цифры.
3. До кризиса 2008 реальная з/п росла выше инфляции, несмотря на ее высокий уровень. После 2008 реальная з/п обычно снижалась вслед за достижением двузначного уровня инфляции.